# Pronova LLM Run Model #
## Use this notebook to do the following ##
- Run the current model on a query
- start a flask api server that accepts a query and will return a response

In [1]:
# Load require libraries
import os
from qdrant_client import QdrantClient
from qdrant_client.http import models
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display

# Load environment variables from .env file
load_dotenv()

True

### Setup Qdrant connection ###

In [2]:
# Get the Qdrant API key from the environment variable
Qdrant_api_key = os.getenv('Qdrant_API_KEY')
if not Qdrant_api_key:
    raise ValueError("No Qdrant API key found in environment variables")
Qdrant_url = os.getenv('Qdrant_URL')
if not Qdrant_url:
    raise ValueError("No Qdrant URL found in environment variables")


# Initialize Qdrant client
try:
    Qclient = QdrantClient(
        url= Qdrant_url,
        api_key=Qdrant_api_key
    )
    print("Successfully connected to Qdrant")
except Exception as e:
    print(f"Failed to connect to Qdrant: {e}")
    raise

Successfully connected to Qdrant


### Setup OpenAI connection ###

In [3]:
# Get the OpenAI API key from the environment variable
OpenAI_api_key = os.getenv('OPENAI_API_KEY')
if not OpenAI_api_key:
    raise ValueError("No OpenAI API key found in environment variables")

OpenAI.api_key = OpenAI_api_key

### Get an OpenAI embedding from a text segment (Function) ###

In [4]:
# Function to get the embedding of a text
def get_embedding(text):
    client = OpenAI()
    response = client.embeddings.create(
        model="text-embedding-ada-002",
        input=text
    )
    return response.data[0].embedding

### Retrieve similar chunks from query (Function) ###

In [5]:
def retrieve_relevant_chunks(collection_name, query, top_k=10):
    query_embedding = get_embedding(query)
    
    search_result = Qclient.search(
        collection_name=collection_name,
        query_vector=query_embedding,
        limit=top_k
    )

    contexts = [result.payload["text"] for result in search_result]
    files = [result.payload.get("source_file") for result in search_result]
    
    return contexts, files


### Rank response source importance (Function) ###

In [6]:
from collections import Counter

def file_ratios(files):
    total_files = len(files)
    counts = Counter(files)
    return {file: count*100 / total_files for file, count in counts.items()}


# file_ratios(["a", "a", "b", "c"])
# {'a': 0.5, 'b': 0.25, 'c': 0.25}

### Markdown Print Function ###

In [7]:
def print_markdown(md_text):
    display(Markdown(md_text))

### Generate Response from Query (Function) ###

In [8]:
import numpy as np

def generate_response(collection_name, query, all_query, all_context, all_responses, all_files):
    print("Generating response for query:", query)
    # generate context for new query
    context, files = retrieve_relevant_chunks(collection_name, query)
    
    # print("Adding new files: ", files)
    files_used = np.unique(files).tolist()
    # files_used = file_ratios(files_used)

    system_role = "You are a specialized assistant that only provides advice on dog-related veterinary care. If a user asks about any other animal or topic outside of dog health, politely decline to answer and remind them that you only provide information about dogs. You will always start by asking the user their dog's name, age, and breed if they didn't already provide it."
    # Combine retrieved chunks into a single string
    context_text = "\n".join(context)

    # append query and context to the running lists
    all_query.append(query)
    all_context.append(context_text)
    all_files.append(files_used)

    # print("All files used now:", all_files)
    # create the messages object using all the queries and contexts
    messages = [{"role": "system", "content": system_role}]

    for i in range(len(all_query)):
        messages.append({"role": "system", "content": "Use this context to answer my following question: " + all_context[i]})
        messages.append({"role": "user", "content": all_query[i]})
        if i < len(all_responses):
            messages.append({"role": "system", "content": all_responses[i]})
    
    # print(messages)


    # Generate a response using GPT-4
    client = OpenAI()
    completion = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=messages
    )
    all_responses.append(completion.choices[0].message.content)
    return all_query, all_context, all_responses, all_files


### Playground (use this to test querys in the notebook)

In [12]:
collection_name = "FullModel"
query = "My dog Bamba, a 5yo golden, has been limping after short walks."
query_list, context_list, response_list, files_list = generate_response(collection_name, query, [], [], [], [])


# # Qclient.get_collections()
print(response_list[0])
# print([file for file in files_list[0]])
for file in files_list[0]:
    # print(file)
    cite = get_citation(file)
    print(cite)


Generating response for query: My dog Bamba, a 5yo golden, has been limping after short walks.
I'm sorry to hear that Bamba is limping. Limping can be a sign of pain or discomfort, and it's important to assess the severity of the situation. Since Bamba is limping after short walks, I recommend keeping her activity level low and providing her with plenty of rest. 

If her limp persists or she shows any other signs of pain, such as reluctance to bear weight on the leg, whining, or changes in behavior, it's best to make an appointment with your veterinarian. They can evaluate her properly and determine the underlying cause of her limping.

Please avoid giving any medications at home until you’ve consulted with the vet, as some might not be safe for dogs without a prescription. Make sure to monitor her condition closely and seek veterinary advice as soon as possible to ensure that Bamba gets the appropriate care.
(' Dog Tail Injury: Signs and Causes', 'https://www.petmd.com//dog/conditions

In [10]:
import json

def get_citation(filename):
    files_list = ["scrapingDemo/sources_petMD_allergies.json", "scrapingDemo/sources_petMD_behavior.json", "scrapingDemo/sources_petMD_nutrition.json", "scrapingDemo/sources_petMD_care_healthy_living.json", "scrapingDemo/sources_petMD_procedures.json", "scrapingDemo/sources_petMD_symptoms.json", "scrapingDemo/sources_petMD.json"]
    citation = {}
    
    for file in files_list:
        with open(file, 'r', encoding='utf-8') as f:
            sources = json.load(f)
            if filename in sources:
                citation = sources[filename]
                break
    
    url = citation.get('URL', 'URL not found')
    author = citation.get('Author', 'Author not found')
    date = citation.get('Date', 'Date not found')
    topic = citation.get('Topic', 'Topic not found')
    
    
    return topic, url, author, date


### Lightweight Flask Server (for Frontend API testing) ###

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS

collection_name = "FullModel" # for eyals db
# collection_name = "LLM_V1"  # for other db


app = Flask(__name__)
CORS(app)  # This will enable CORS for all routes

@app.route('/query', methods=['POST'])
def query_llm():
    data = request.json
    new_query = data.get('new_query')
    queries = data.get('queries')
    contexts = data.get('contexts')
    responses = data.get('responses')
    
    # if not new_query or not queries or not contexts or not responses:
    #     return jsonify({'error': 'New query, queries, contexts, and responses must be provided'}), 400

    try:
        updated_queries, updated_contexts, updated_responses = generate_response(collection_name, new_query, queries, contexts, responses)
        return jsonify({
            'queries': updated_queries,
            'contexts': updated_contexts,
            'responses': updated_responses
        })
    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)